In [1]:
from datetime import timedelta
import numpy as np
import polars as pl
import pandas as pd
import implicit

from tools import load_data_actions, generate_lightfm_recs_mapper
from tqdm import tqdm

from lightfm.data import Dataset
from lightfm import LightFM

from rectools import Columns
from rectools.dataset import Dataset as RTDataset
from rectools.models import (
    ImplicitALSWrapperModel,
    ImplicitBPRWrapperModel,
    LightFMWrapperModel,
    PureSVDModel,
    ImplicitItemKNNWrapperModel,
    EASEModel
)

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import roc_auc_score

/opt/anaconda3/envs/recsys/lib/python3.11/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [2]:
def dataframe2rectools(df):
    return (
        df[["cookie", "node", "event_date"]]
        .with_columns(pl.lit(1).alias('weight'))
        .rename({
            "cookie": Columns.User,
            "node": Columns.Item,
            "weight": Columns.Weight,
            "event_date": Columns.Datetime,
        })
    ).to_pandas()

In [3]:
SEED = 42
top_N = 40
DATA_DIR = 'data/'

df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features_preproc_20.pq')
df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

df_train, df_eval = load_data_actions(df_clickstream, df_event)

mean_by_cat = (
    df_text_features
    .join(df_cat_features["item", "node"], on="item", how="left")
    .to_pandas()
    .groupby("node")["title_projection"]
    .apply(lambda vs: np.mean(np.stack(vs.values), axis=0).tolist())
    .reset_index(name="mean_title_projection")
)

addit_features = pl.read_parquet(DATA_DIR+"cat_features_preproc_20.pq")
atr_cols = [i for i in addit_features.columns if "attr" in i]
addit_features = (
    addit_features.group_by("node")
    .agg([pl.col(atr).mean() for atr in atr_cols])
)

item_features = pl.DataFrame(mean_by_cat)
item_features = item_features.with_columns(pl.col("mean_title_projection").list.to_struct()).unnest("mean_title_projection")
item_features = item_features.join(addit_features, how="left", on="node")
item_features = item_features.rename({"node": "id"})
item_features = item_features.rename({"id": "node"})

df_train = dataframe2rectools(df_train)

/var/folders/1v/2njvtrl903s020_gk0tw647c0000gn/T/ipykernel_24092/3680953557.py:31: UserWarning: `to_struct()` should be passed a list of field names to avoid query errors in subsequent operations (e.g. <struct operation> not supported for dtype Unknown)
  item_features = item_features.with_columns(pl.col("mean_title_projection").list.to_struct()).unnest("mean_title_projection")


In [4]:
candidates_ALS = pl.read_csv('data/first_stage_candidates_ALS_200.csv')
candidates_ANN = pl.read_csv('data/first_stage_prediction_ANN_40.csv')
candidates_BM25 = pl.read_csv('data/first_stage_prediction_BM25_40.csv')

In [5]:
candidates = pl.concat([candidates_ALS, candidates_BM25, candidates_ANN])
candidates = candidates.unique(subset=["cookie", "node"], maintain_order=True)
candidates = candidates.to_pandas()
candidates['rank'] = candidates.groupby('cookie').cumcount() + 1 

In [6]:
df_eval = df_eval.to_pandas()
df_eval = df_eval.astype({"node": int})

In [36]:
# positive interactions
pos = candidates.merge(
    df_eval,
    on=['cookie', 'node'],
    how='inner'
    )

pos.loc[:, 'target'] = 1
print(pos.shape)
# negative interactions
neg = candidates.set_index(['cookie', 'node'])\
        .join(df_eval.set_index(['cookie', 'node']))

neg = neg[neg['event'].isnull()].reset_index()     
neg = neg.sample(frac=0.03)
neg.loc[:, 'target'] = 0

print(neg.shape)


# train test
ctb_train_users, ctb_test_users = train_test_split(
    df_eval['cookie'].unique(),
    random_state=SEED,
    test_size=0.2
    )
# train eval
# ctb_train_users, ctb_eval_users = train_test_split(
#     ctb_train_users,
#     random_state=SEED,
#     test_size=0.1
#     )

select_col = ["cookie", "node", "rank", 'target']


# Catboost train
ctb_train = shuffle(
    pd.concat([
        pos[pos['cookie'].isin(ctb_train_users)],
        neg[neg['cookie'].isin(ctb_train_users)]
])[select_col]
)
# Catboost test
ctb_test = shuffle(
    pd.concat([
        pos[pos['cookie'].isin(ctb_test_users)],
        neg[neg['cookie'].isin(ctb_test_users)]
])[select_col]
)
# # for early stopping
# ctb_eval = shuffle(
#     pd.concat([
#         pos[pos['cookie'].isin(ctb_eval_users)],
#         neg[neg['cookie'].isin(ctb_eval_users)]
# ])[select_col]
# )

(47261, 5)
(413161, 5)


In [37]:
ctb_train['target'].value_counts(normalize=True), ctb_test['target'].value_counts(normalize=True)

(target
 0    0.897363
 1    0.102637
 Name: proportion, dtype: float64,
 target
 0    0.897314
 1    0.102686
 Name: proportion, dtype: float64)

In [40]:
item_features = item_features.to_pandas()
user_col = ['cookie']
item_col = item_features.columns

train_feat = (
    ctb_train
    # .merge(
    #     users[user_col],
    #     on=['user_id'],
    #     how='left')
    .merge(
        item_features[item_col],
        on=['node'],
        how='left')
)
# eval_feat = (
#     ctb_eval
#     # .merge(
#     #     users[user_col],
#     #     on=['user_id'],
#     #     how='left')
#     .merge(
#         item_features[item_col],
#         on=['node'],
#         how='left')
# )
test_feat = (
    ctb_test
    # .merge(
    #     users[user_col],
    #     on=['user_id'],
    #     how='left')
    .merge(
        item_features[item_col],
        on=['node'],
        how='left'
        )
)
train_feat.isna().sum().sum(), eval_feat.isna().sum().sum()

AttributeError: 'DataFrame' object has no attribute 'to_pandas'

In [41]:
train_feat = train_feat.sort_values(by=["cookie", "target", "rank"], ascending=[True, False, True])
# eval_feat = eval_feat.sort_values(by=["cookie", "target", "rank"], ascending=[True, False, True])
test_feat = test_feat.sort_values(by=["cookie", "target", "rank"], ascending=[True, False, True])

In [42]:
drop_col = ['cookie', 'node']
target_col = ['target']
cat_col = [f"field_{i}" for i in range(64)]

X_train, y_train = train_feat.drop(drop_col + target_col, axis=1), train_feat[target_col]
train_cookies = train_feat["cookie"]
# X_val, y_val = eval_feat.drop(drop_col + target_col, axis=1), eval_feat[target_col]
# val_cookies = eval_feat["cookie"]
X_test, y_test = test_feat.drop(drop_col + target_col, axis=1), test_feat['target']
test_cookies = test_feat["cookie"]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((368345, 85), (368345, 1), (92077, 85), (92077,))

In [101]:
Pool.__init__

<function catboost.core.Pool.__init__(self, data, label=None, cat_features=None, text_features=None, embedding_features=None, embedding_features_data=None, column_description=None, pairs=None, graph=None, delimiter='\t', has_header=False, ignore_csv_quoting=False, weight=None, group_id=None, group_weight=None, subgroup_id=None, pairs_weight=None, baseline=None, timestamp=None, feature_names=None, feature_tags=None, thread_count=-1, log_cout=None, log_cerr=None, data_can_be_none=False)>

In [104]:
from catboost import Pool
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=train_cookies,
    cat_features=["rank"]
)
# val_pool = Pool(
#     data=X_val,
#     label=y_val,
#     group_id=val_cookies
# )
test_pool = Pool(
    data=X_test,
    label=y_test,
    group_id=test_cookies,
    cat_features=["rank"]
)

In [114]:
from catboost import CatBoostClassifier, CatBoostRanker
from copy import deepcopy

# параметры для обучения
est_params = {
  'subsample': 0.9,
  'max_depth': 4,
  'n_estimators': 5000,
  'learning_rate': 0.03,
  'thread_count': 20,
  'random_state': SEED,
  'verbose': 200,
#   "loss_function":'YetiRankPairwise',
#  "eval_metric":'NDCG'
}

ctb_model = CatBoostClassifier(**est_params)


default_parameters = {
    'iterations': 5000,
    'custom_metric': ['NDCG', 'AUC'],
    'verbose': 100,
    # "loss_function":'YetiRankPairwise',
    'thread_count': 20,
    'learning_rate': 0.1,
    'max_depth': 4,
    'random_seed': SEED,
}

parameters = {}

def fit_model(loss_function, additional_params=None, train_pool=None, test_pool=None):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function

    if additional_params is not None:
        parameters.update(additional_params)

    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=True, early_stopping_rounds=100,)

    return model

In [118]:
model = fit_model(
    'PairLogitPairwise', 
    {'custom_metric': ['PrecisionAt:top=40', 'RecallAt:top=40', 'MAP:top=40', 'AUC']},
    train_pool=train_pool,
    test_pool=test_pool
    )

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Pairwise scoring loss functions on CPU do not support one hot features. OneHotMaxSize set to 1
0:	learn: 0.6680991	test: 0.6678398	best: 0.6678398 (0)	total: 442ms	remaining: 36m 49s
100:	learn: 0.5240137	test: 0.5274091	best: 0.5274091 (100)	total: 45.4s	remaining: 36m 41s


KeyboardInterrupt: 

In [71]:
0.7585 # RMSE

0.7585

In [72]:
y_pred = model.predict(X_test)
print(f"ROC AUC score = {roc_auc_score(y_test, y_pred):.4f}")

ROC AUC score = 0.7585


In [73]:
# # save model 
import dill 
with open(f"ctb_model_ranker.dill", 'wb') as f:
    dill.dump(ctb_model, f)

# Submission

In [ ]:
# from catboost import CatBoostClassifier, CatBoostRanker

In [74]:
# import dill
# with open("ctb_model_baseline.dill", "rb") as f:
#     ctb_model = dill.load(f)

In [75]:
df_final = dataframe2rectools(df_clickstream[["cookie", "node", "event_date"]])

In [76]:
candidates_ALS = pl.read_csv('data/test_hybrid_stage_candidates_ALS_200.csv')
candidates_ANN = pl.read_csv('data/test_prediction_ANN_40.csv')
candidates_BM25 = pl.read_csv('data/test_hybrid_stage_candidates_BM25_40.csv')

In [77]:
candidates = pl.concat([candidates_ALS, candidates_BM25, candidates_ANN])
candidates = candidates.unique(subset=["cookie", "node"], maintain_order=True)
candidates = candidates.to_pandas()
candidates['rank'] = candidates.groupby('cookie').cumcount() + 1 

In [78]:
candidates.shape

(23032214, 3)

In [80]:
user_col = ['cookie']
drop_col = ['cookie', 'node']
target_col = ['target']
cat_col = [f"field_{i}" for i in range(64)]
item_col = item_features.columns
item_features = item_features.to_pandas()
final_feat = (
    candidates
    # .merge(
    #     users[user_col],
    #     on=['user_id'],
    #     how='left')
    .merge(
        item_features[item_col],
        on=['node'],
        how='left')
)
X_final = final_feat.drop(drop_col, axis=1)

In [81]:
del item_features
del candidates

In [83]:
y_pred = model.predict(X_final)

In [84]:
del X_final

In [86]:
final_feat["score"] = y_pred
res = final_feat.sort_values(by=["cookie", "score"], ascending=[True, False]).groupby("cookie").head(40)
res = res[["cookie", "node"]].drop_duplicates()
res = pl.DataFrame(res)
addit = df_test_users.filter(pl.col("cookie").is_in(res["cookie"]) == False).with_columns(pl.lit(1).cast(pl.Int64).alias('node'))
addit

cookie,node
i64,i64


In [87]:
res.head()

cookie,node
i64,i64
1,57915
1,239954
1,198003
1,195257
1,199289


In [88]:
pl.concat([res, addit]).group_by("cookie").head(40)["cookie","node"].write_csv('results/prediction_hybrid_ranker_final.csv')